In [ ]:

I have a table with a varchar column, and I would like to find all the records that have duplicate values in this 

column. What is the best query I can use to find the duplicates?





In [ ]:
Do a SELECT with a GROUP BY clause. Lets say name is the column you want to find duplicates in:

SELECT name, COUNT(*) c 
FROM table 
GROUP BY name 
HAVING c > 1;


This will return a result with the name value in the first column, and a count of how many times that value appears 
in the second.



In [ ]:
Alternative solution:
    
SELECT varchar_col
FROM table
GROUP BY varchar_col
HAVING COUNT(*) > 1;



In [ ]:


I want to pull out duplicate records in a MySQL Database. This can be done with:

SELECT address, count(id) as cnt FROM list
GROUP BY address HAVING cnt > 1
Which results in:

100 MAIN ST    2

I would like to pull it so that it shows each row that is a duplicate. Something like:

JIM    JONES    100 MAIN ST
JOHN   SMITH    100 MAIN ST



In [ ]:
SELECT firstname, 
   lastname, 
   list.address 

FROM list
   
    INNER JOIN (SELECT address
               FROM   list
               GROUP  BY address
               HAVING COUNT(id) > 1) dup
    
           ON list.address = dup.address;

In [ ]:
Difference between TRUNCATE, DELETE and DROP commands?

The DELETE command is used to remove some or all rows from a table. A WHERE clause can be used to only remove some 
rows. If no WHERE condition is specified, all rows will be removed. After performing a DELETE operation you need to COMMIT or ROLLBACK the transaction to make the change permanent or to undo it. Note that this operation will cause all DELETE triggers on the table to fire.

SQL> SELECT COUNT(*) FROM emp;
  COUNT(*)
----------
        14

SQL> DELETE FROM emp WHERE job = 'CLERK';
4 rows deleted.

------------------------------------------------------------------------------------------------------------------------

SQL> COMMIT;
Commit complete.

SQL> SELECT COUNT(*) FROM emp;
  COUNT(*)
----------
        10
    
------------------------------------------------------------------------------------------------------------------------

TRUNCATE removes all rows from a table. The operation cannot be rolled back and no triggers will be fired. As such, 
TRUNCATE is faster and doesn't use as much undo space as a DELETE.

SQL> TRUNCATE TABLE emp;
Table truncated.

SQL> SELECT COUNT(*) FROM emp;

  COUNT(*)
----------
         0
    
------------------------------------------------------------------------------------------------------------------------

The DROP command removes a table from the database. All the tables' rows, indexes and privileges will also be removed. 
No DML triggers will be fired. The operation cannot be rolled back.

SQL> DROP TABLE emp;
Table dropped.

SQL> SELECT * FROM emp;
SELECT * FROM emp
              *
ERROR at line 1:
ORA-00942: table or view does not exist
    
------------------------------------------------------------------------------------------------------------------------


DROP and TRUNCATE are DDL commands, whereas DELETE is a DML command. Therefore DELETE operations can be rolled back 
(undone), while DROP and TRUNCATE operations cannot be rolled back.


PS: DELETE will not free up used space within a table. This means that repeated DELETE commands will severely fragment the table and queries will have to navigate this "free space" in order to retrieve rows.



In [ ]:
How to delete duplicate records in MySQL, in a table without IDs?

I need to delete the duplicate records in this table. However, there is no id for each row.

Example Data
+---------+--------+----------+
| product | amount | quantity |
+---------+--------+----------+
| table   |   2000 |        5 |
| chair   |    300 |       25 |
| TV      |  30000 |        4 |
| bike    |    300 |       25 |
| table   |   2000 |        5 |
| chair   |    300 |       25 |
| chair   |    300 |       25 |
+---------+--------+----------+

Expected Results

I need to get this result.

+---------+--------+----------+
| product | amount | quantity |
+---------+--------+----------+
| table   |   2000 |        5 |
| chair   |    300 |       25 |
| TV      |  30000 |        4 |
| bike    |    300 |       25 |
+---------+--------+----------+

Script with ID
If there were an id, I could have used:

DELETE p1 FROM products p1
INNER JOIN products p2 
WHERE p1.id < p2.id AND p1.product = p2.product;

------------------------------------------------------------------------------------------------------------


First solution: move unique records to a copy of table and replace original table.

CREATE TABLE temp LIKE products;
INSERT INTO temp 
    SELECT DISTINCT * FROM products;
DROP TABLE products;
RENAME TABLE temp TO products;
Second solution: add temporary autoincrement, delete records using it, and drop temp field.
    
------------------------------------------------------------------------------------------------------------

ALTER TABLE products ADD COLUMN temp SERIAL;


DELETE t1.* 
    FROM products t1 
    LEFT JOIN ( SELECT MIN(temp) mintemp 
                FROM products
                GROUP BY field1,field2 /* , ... */ , fieldN) t2 
        ON t1.temp=t2.mintemp 
    WHERE t2.mintemp IS NULL;
ALTER TABLE products DROP COLUMN temp;

